In [1]:
import ase
import ase.build
from ase.db import connect
import itertools
from matplotlib import pyplot as plt
import numpy as np
from openbabel import openbabel as ob
import os
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from schnetpack import Properties
import sys
from typing import Tuple

sys.path.append('..')
sys.path.append('../analyses')
import analyses.analysis as analysis

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [ ]:
with open('../analyses/molecules/generated/v6/nequip/interactions=4/')

In [4]:
generated_db = '../analyses/molecules/generated/v5/nequip/interactions=4/l=5/channels=32/beta=1.0/step=best/generated_molecules.db'
generated_dists = []

with connect(generated_db) as conn:
    for row in conn.select():
        mol = row.toatoms()
        for atom1, atom2 in itertools.combinations(mol.positions, 2):
            generated_dists.append(np.linalg.norm(atom1 - atom2))

training_dists = []

model, params, config = analysis.load_model_at_step(
    '../workdirs/v6/nequip/interactions=4/l=5/channels=32', -1, run_in_evaluation_mode=True
)

with connect('../qm9_data/qm9-all.db') as conn:
    i = 0
    for row in conn.select():
        mol = row.toatoms()
        for atom1, atom2 in itertools.combinations(mol.positions, 2):
            training_dists.append(np.linalg.norm(atom1 - atom2))
        if i > 100:
            break